# Exploring behavior of envelope proteins

In [1]:
import numpy as np 
import pandas as pd 
import altair as alt
import futileprot.viz
import altair_saver 
colors, palette = futileprot.viz.altair_style()

In [3]:
data = pd.read_csv('../../data/literature/Belliveau2021/Belliveau2021_absolute_measurements.csv')

In [13]:
# Use go terms to classify
classes = {'outer membrane': 'GO:0009279',
           'inner membrane': 'GO:0005886',
           'periplasm': 'GO:0042597', 
           'cytosol': 'GO:0005829'}

df = pd.DataFrame([])
for g, d in data.groupby(['dataset', 'dataset_name', 'condition', 'growth_rate_hr']):
    tot_prot = d['fg_per_cell'].sum()
    for k, v in classes.items():
        class_prot = d[d['go_terms'].str.contains(v)]['fg_per_cell'].sum()
        df = df.append({'dataset_name':g[1],
                        'condition': g[2],
                        'growth_rate_hr': g[3], 
                        'location': k,
                        'location_mass': class_prot,
                        'total_protein_mass': tot_prot, 
                        'location_frac': class_prot / tot_prot
                        }, ignore_index=True)

In [17]:
out = alt.Chart(df[df['location']!='cytosol']).mark_point().encode(
        x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'), 
        y=alt.Y('location_frac:Q', title='mass fraction'), 
        color=alt.Color('location:N', title='localization'),
        shape=alt.Shape('dataset_name:N', title='dataset')
)
altair_saver.save(out, './localization_mass_fraction.pdf')

In [32]:
summed = df[(df['location']!='cytosol') & 
            (df['location'] != 'inner membrane')].groupby(['dataset_name', 'condition', 'growth_rate_hr']).sum().reset_index()

In [33]:
out = alt.Chart(summed).mark_point().encode(
        x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'), 
        y=alt.Y('location_frac:Q', title='mass fraction'), 
        color=alt.Color('dataset_name:N', title='dataset')
)

In [34]:
out = out.properties(title='cell envelope (outer membrane + periplasm)')
altair_saver.save(out, './envelope_mass_fraction.pdf')


In [35]:
summed = df[(df['location']!='cytosol')].groupby(['dataset_name', 'growth_rate_hr']).sum().reset_index()
out = alt.Chart(summed).mark_point().encode(
        x=alt.X('growth_rate_hr:Q', title='growth rate [per hr]'), 
        y=alt.Y('location_frac:Q', title='mass fraction'), 
        color=alt.Color('dataset_name:N', title='dataset')
)
out = out.properties(title='cell periphery (outer membrane + periplasm + inner membrane)')
altair_saver.save(out, './periphery_mass_fraction.pdf')
